# How to refine a functional map?

In [1]:
import geomstats.backend as gs

from geomfum.dataset import NotebooksDataset
from geomfum.refine import (
    FastSinkhornFilters,
    IcpRefiner,
    ZoomOut,
)
from geomfum.shape import TriangleMesh

[Load meshes](load_mesh_from_file.ipynb).

In [2]:
dataset = NotebooksDataset()

mesh_a = TriangleMesh.from_file(dataset.get_filename("cat-00"))
mesh_b = TriangleMesh.from_file(dataset.get_filename("lion-00"))

mesh_a.n_vertices, mesh_b.n_vertices

(7207, 5000)

[Set Laplace eigenbasis](./mesh_laplacian_spectrum.ipynb) for each mesh.

In [3]:
mesh_a.laplacian.find_spectrum(spectrum_size=10, set_as_basis=True)
mesh_b.laplacian.find_spectrum(spectrum_size=10, set_as_basis=True)

mesh_a.basis.use_k = 6
mesh_b.basis.use_k = 5

Assume we have a [valid functional map](./functional_map.ipynb) $C$ between `mesh_a` and `mesh_b` (which for demonstration purposes, we instantiate randomly).

In [4]:
fmap_matrix = gs.random.uniform(
    size=(mesh_b.basis.spectrum_size, mesh_a.basis.spectrum_size)
)

fmap_matrix.shape

(5, 6)

Refine functional map using ICP algorithm (see [OCSBG2012](https://dl.acm.org/doi/10.1145/2185520.2185526)).

In [5]:
icp = IcpRefiner(nit=10, atol=1e-4)

icp_fmap_matrix = icp(fmap_matrix, mesh_a.basis, mesh_b.basis)

icp_fmap_matrix.shape

(5, 6)

Refine functional map using ZoomOut (see [MRRSWO2019](https://arxiv.org/abs/1904.07865)).

In [6]:
zoomout = ZoomOut(nit=2, step=(2, 1))

zoomout_fmap_matrix_ = zoomout(fmap_matrix, mesh_a.basis, mesh_b.basis)

zoomout_fmap_matrix_.shape

(7, 10)

Refine functional map using fast Sinkhorn filters (see [PRMWO2021](https://openaccess.thecvf.com/content/CVPR2021/html/Pai_Fast_Sinkhorn_Filters_Using_Matrix_Scaling_for_Non-Rigid_Shape_Correspondence_CVPR_2021_paper.html)).

In [7]:
fast_sinkhorn_filters = FastSinkhornFilters(nit=2, step=(2, 1))

fast_sinkhorn_filters_fmap_matrix_ = fast_sinkhorn_filters(
    fmap_matrix, mesh_a.basis, mesh_b.basis
)

fast_sinkhorn_filters_fmap_matrix_.shape

[KeOps] Warning : cuda was detected, but driver API could not be initialized. Switching to cpu only.


(7, 10)

[Pointwise](./pointwise_from_functional.ipynb) and [functional](./functional_from_pointwise.ipynb) converters can be easily controlled through `p2p_from_fm_converter` and `fm_from_p2p_converter` (sensible defaults are used.)

## Further reading

* [How to compute a functional map?](./functional_map.ipynb)

* [How to compute a pointwise map from a functional map?](./pointwise_from_functional.ipynb)

* [How to compute a functional map from a pointwise map?](./functional_from_pointwise.ipynb)